-sandbox
# Multistep Workflows

Machine learning projects quickly become complex when additional steps to the training process are added.  This lesson examines managing the complexity of multistep machine learning projects using multistep workflows.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - Introduce multistep workflows
 - Execute a single, parameterized notebook as a step in a pipeline
 - Execute a multistep workflow

-sandbox
### Managing the Complexity

Tools like Scikit-Learn and Spark ML use the abstraction of pipelines that bundle multiple stages together into a single object.  This works well as a first step in handling more complexity in the machine learning training process but has limitations in that these pipelines don't scale to support arbitrary code.  Rather, they normally only work with built-in functions in those packages.  Additionally, they do not allow for monitoring of each individual stage in that process.

MLflow allows for more robust pipelining by using multistep workflows.  This not only allows for a more flexible API but it allows teams to collaborate at higher level since different contributors can be responsible for different aspects of the pipeline.

**Multistep pipelines scales MLflow projects by allowing each stage to be its own project.**  The underlying idea that makes this possible is that **runs can recursively call other runs.**  This means that steps in a machine learning pipeline can be isolated.

There are a few options for passing artifacts between runs.  They can be saved to a common location or they can be logged as an artifact within MLflow.  This also allows for quicker iteration since runs can be run individually, you only need to rerun a given step if the last step's results have been cached.

Below is an example of a multistep workflow with four distinct steps.  <a href="https://github.com/mlflow/mlflow/tree/master/examples/multistep_workflow" target="_blank">You can see the full repository here.</a>

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-4/mlproject-architecture0.png" style="height: 250px; margin: 20px"/></div>

-sandbox
### Multistep Architecture

Now that we can package projects and run them in their environment, let's look at how  we can make workflows consisting of multiple steps.    There are three general architectures to consider:<br><br>

1. One driver project calls other entry points in that same project
2. One driver project calls other projects 
3. One project calls another project as its final step

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-4/mlproject-architecture1.png" style="height: 250px; margin: 20px"/></div>
<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-4/mlproject-architecture2.png" style="height: 250px; margin: 20px"/></div>
<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-4/mlproject-architecture3.png" style="height: 250px; margin: 20px"/></div>

In [0]:
%run "./Includes/Classroom-Setup"

In the last lesson, we used a Python file as a project that we ran.  In this lesson, we'll use Databricks notebooks instead.  These are all under the `Multistep` folder in your directory of notebooks.  There are two things to note about notebooks:<br><br>

0. Notebooks can be run using `dbutils.notebook.run("<path>", "<timeout>")`
0. Notebooks can be parameterized using widgets

Run the following code to see how this works.

In [0]:
step1 = dbutils.notebook.run("./Multistep/Step-1-Read-Data", 60, 
  {"data_input_path": "/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv"})

print(step1)

This is the first step of our multistep workflow.  From the above, you can see that we ran the notebook `Step-1-Read-Data` in the `Multistep` directory.  We had a timeout of 60 seconds so if the code doesn't complete in that time, it would have failed.  Finally, we have the input path for our data.

-sandbox
Take a look at the underlying notebook.  Note the following is accomplished in four cells:<br><br>

0. Widgets are declared.  If you run these cells in the notebook itself, you'll see widgets appear at the top of the screen.  Widgets allow for the customization of notebooks without editing the code itself. They also allow for passing parameters into notebooks. 
0. Widgets are read.  This allows you to get the value of the widget and use it in your code.
0. An MLflow run logs the data from `data_input_path` as an artifact
0. The notebook exits and reports back information to the parent notebook (in our case, that's this notebook).

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Check out <a href="https://docs.databricks.com/user-guide/notebooks/widgets.html" target="_blank">the Databricks documentation on widgets for additional information </a>

-sandbox
We can get data back from the executed notebook by parsing the exit string.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> For debugging, you can either write your code in a notebook and then execute it as a run or you can click on `Notebook job #XXX` that appears after using `dbutils.notebook.run` to see where any problems might be.

In [0]:
import json

data_output_path = json.loads(step1).get("data_output_path")

print(data_output_path)

### Multistep Workflow

Now that we've created a single executable notebook with input parameters and output data, we can create more complex workflows.  [Take a look at the second step in our workflow.]($./Multistep/Step-2-Train)  This notebook takes the data logged as an artifact and trains a model using specific hyperparameters.  The trained model is also logged.

In [0]:
step2 = dbutils.notebook.run("./Multistep/Step-2-Train", 60, 
  {"artifact_URI": data_output_path,
   "n_estimators": 10,
   "max_depth": 20,
   "max_features": "auto"})

print(step2)

Get the model output path from the result.

In [0]:
model_output_path = json.loads(step2).get("model_output_path")

print(model_output_path)

Now [take a look at the final step in our workflow.]($./Multistep/Step-3-Predict)  This step takes the saved model and creates predictions from it.  It logs those predictions as an artifact.

In [0]:
step3 = dbutils.notebook.run("./Multistep/Step-3-Predict", 60, 
  {"model_path": model_output_path,
   "data_path": data_output_path})

print(step3)

Parse the notebook output to see the predictions

In [0]:
predictions_output_path = json.loads(step3).get("predictions_output_path")

print(dbutils.fs.head(dbutils.fs.ls(predictions_output_path)[0].path))

## Review

**Question:** What are the benefits of pipelining?  
**Answer:** The biggest benefit to pipelining is managing the complexity of the machine learning process.  With loading, ETL, featurization, training, and prediction stages--to name a few--the complexity of the process quickly grows.  One additional benefit is collaboration since different individuals can work on different stages in the pipeline.

**Question:** How can I manage a pipeline using MLflow?  
**Answer:** Multi-step workflows chain together multiple MLflow jobs.  Runs can recursively call other runs.  This allows each stage to be its own MLflow project with its own environment, inputs, and outputs.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Cleanup<br>

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [0]:
%run "./Includes/Classroom-Cleanup"

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Next Steps

Start the labs for this lesson, [Multistep Workflows Lab]($./Labs/04-Lab)

## Additional Topics & Resources

**Q:** Where can I find out more information on the future of Multistep Workflows?  
**A:** Check out Aaron Davidson's demo <a href="https://databricks.com/session/accelerating-the-machine-learning-lifecycle-with-mlflow-1-0" target="_blank">at Spark Summit 2019 for the direction of the project.</a>

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>